### Simple Moving Average

Moving average is the calculating to analyze data points by creating a series of averages of different sections of the full data set.

Simple Moving Average (**SMA**) is the unweighted mean of the previous _k_ data points.

$$ SMA_k = \frac{1}{k} \sum_{i=n-k+1}^n p_i $$

$$ p = {Error \over Production} $$

$$ MovingAverage = \frac{1}{k} \sum_{i=n-k+1}^n {Error_i \over Production_i} $$

In [1]:
!pip install -r interpolate-requirements.txt

In [2]:
import pandas as pd
import numpy as np
import pika
import json
import sql_operations
import math

In [3]:
# Temporary for in operation
import psycopg2
import sqls

In [4]:
mq_connection = pika.BlockingConnection(
    pika.ConnectionParameters('172.16.238.20'))
channel = mq_connection.channel()
channel.queue_declare(queue='new_interpolarate')

<METHOD(['channel_number=1', 'frame_type=1', "method=<Queue.DeclareOk(['consumer_count=0', 'message_count=1', 'queue=new_interpolarate'])>"])>

In [5]:
connection = psycopg2.connect(database='postgres_db',
                              user='postgres',
                              password='p.postgres',
                              host='172.16.238.10',
                              port=5432)
cursor = connection.cursor()

In [6]:
boxing_size = 1000

In [7]:
def get_tag_data_frame(ids):
    cursor.execute(sqls.get_interpolated_tags, (tuple(ids),))
    interpolated_data = cursor.fetchall()
    interpolated = []
    for data in interpolated_data:
        interpolated.append({'timestamp': data[0], 'value': data[1]})
    return pd.DataFrame(interpolated) if len(interpolated) > 0 else None

In [8]:
def get_merged_data_frame(master_ids, error_ids):
    master_df = get_tag_data_frame(master_ids)
    error_df = get_tag_data_frame(error_ids)
    merged_df = master_df.merge(error_df, on='timestamp', how='left').fillna(0)
    return merged_df

In [9]:
def save_box(df, production_total, error_total, batch_id, master_tag_id, error_tag_id):
    start_time = df.iloc[0]['Timestamp']
    end_time = df.iloc[len(df) - 1]['Timestamp']
    success_total = df['Success'].sum()
    total_start = production_total - success_total
    total_end = production_total
    error_current_total = df['Error'].sum()
    error_start = error_total - error_current_total
    error_end = error_total
    size = success_total
    error_rate = error_current_total/success_total * 100
    inserted_id = sql_operations.insert_boxed_data(start_time, 
                                                  end_time,
                                                  int(total_start),
                                                  int(total_end),
                                                  int(error_start),
                                                  int(error_end),
                                                  int(size),
                                                  int(error_current_total),
                                                  float(error_rate))
    sql_operations.insert_boxed_batch(inserted_id, batch_id)
    sql_operations.insert_boxed_tag(inserted_id, master_tag_id)
    sql_operations.insert_boxed_tag(inserted_id, error_tag_id)

In [10]:
def find_slice_index(current_boxing, current_total, current_error):
    idx = 0
    diff = 0
    approximate_errors_for_diff = 0
    for index, row in current_boxing.iterrows():
        row_production = row['Success']
        row_error = row['Error']
        current_total -= int(row_production)
        current_error -= int(row_error)
        if current_error < boxing_size:
            idx = index
            diff = boxing_size - current_total
            approximate_errors_for_diff = math.ceil(diff * row_error / row_production)
            current_total += diff
            current_error += approximate_errors_for_diff
            idx += 1
            break
    percentage = 0
    if current_total > 0:
        percentage = current_error/ current_total * 100
    return idx, diff, approximate_errors_for_diff, percentage

In [11]:
def calculate_total_production(current_boxing):
    production = current_boxing['Success'].sum()
    error = current_boxing['Error'].sum()
    percentage = 0
    if production >= boxing_size:
        idx, diff, aprx_error, percentage = find_slice_index(current_boxing, production, error)
        current_boxing = current_boxing.iloc[idx:, :]
        current_boxing.reset_index(drop=True, inplace=True)
    if percentage == 0 and production > 0:
        percentage = error/ production * 100
    return current_boxing, percentage

In [12]:
def box_data(df, batch_id, master_tag_id, error_tag_id):
    temp_df = pd.DataFrame(columns=['Timestamp', 'Success', 'Error', 'Error_Percentage'])
    production_total = 0
    error_total = 0
    for index, r in df.iterrows():
        timestamp = r['timestamp']
        produced = r['value_x']
        error = r['value_y']
        production_total += produced
        error_total += error
        temp_df.loc[len(temp_df)] = [timestamp, produced, error, error/produced *100]
        temp_df, percentage = calculate_total_production(temp_df)
        save_box(temp_df, production_total, error_total, batch_id, master_tag_id, error_tag_id)

In [13]:
def process_message(queue_message):
    merged_df = get_merged_data_frame(
        queue_message['MasterTag']['interpolatedIds'], 
        queue_message['ErrorTag']['interpolatedIds'])
    box_data(merged_df, queue_message['BatchId'], queue_message['MasterTag']['id'], 
        queue_message['ErrorTag']['id'])
    sql_operations.commit()

In [14]:
def message_consumer():
    def callback(ch, method, properties, body):
        queue_message = json.loads(body)
        message = process_message(queue_message)
    channel.basic_consume(queue='new_interpolarate', on_message_callback=callback, auto_ack=True)
    channel.start_consuming()

In [15]:
message_consumer()

KeyboardInterrupt: 

In [16]:
mq_connection.close()